In [31]:
import os
import glob
from pathlib import Path
import pickle


import keras
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import lycon
from lshash.lshash import LSHash
from sklearn.preprocessing import normalize

keras.__version__

'2.2.4'

In [4]:
DATA_PATH = Path('./caltech_images/caltech101/')

glob.glob(str(DATA_PATH) + '/*')[:5]

['caltech_images/caltech101/gerenuk',
 'caltech_images/caltech101/hawksbill',
 'caltech_images/caltech101/headphone',
 'caltech_images/caltech101/ant',
 'caltech_images/caltech101/butterfly']

## Extracting the feature vectors from VGG16 (pretrained on ImageNet)

In [ ]:
batch_size = 16

data_generator = ImageDataGenerator(rescale=1./255,)
net = keras.applications.VGG16(include_top=False, weights='imagenet')

generator = data_generator.flow_from_directory(DATA_PATH,
                                               target_size=(224, 224),
                                               batch_size=batch_size,
                                               class_mode=None,
                                               shuffle=False)


In [100]:
#feature_vectors = net.predict_generator(generator, len(filenames) // batch_size, verbose=1) # use for inference (can take some time)
#filenames = generator.filenames

feature_vectors = np.load('./caltech_images/vgg16_features/features.vgg16.npy')
with open(r"./caltech_images/vgg16_features/filenames.pickle", "rb") as input_file:
    feature_filenames = pickle.load(input_file)

In [101]:
feature_vectors.shape
feature_vectors = feature_vectors.reshape((feature_vectors.shape[0], -1))
feature_vectors = normalize(feature_vectors, axis=1, norm='l2') # normalizing every feature vector
feature_vectors.shape

(9088, 25088)

In [102]:
features_table = dict(zip(feature_filenames, feature_vectors))

## Generating the Hashtable (lshash)

In [103]:
hash_index = LSHash(hash_size=16, input_dim=feature_vectors.shape[-1], num_hashtables=8)

In [104]:
for filename, feature_vec in tqdm_notebook(features_table.items()):
    hash_index.index(feature_vec, extra_data=filename)

## Running queries for similar images

In [110]:
def similar_images(index, results=10):
    if isinstance(index, int):
        response = hash_index.query(features_table[list(features_table.keys())[index]], 
                         num_results=11, distance_func='cosine')
    else:
        response = hash_index.query(features_table[index], 
                         num_results=11, distance_func='cosine')
    images = list()
    for i in range(1, 12):
        img = lycon.load(str(DATA_PATH / response[i-1][0][1]))
        images.append(str(DATA_PATH / response[i-1][0][1]))
    imagesList = ''.join( [f"<img style='height: 120px; margin: 0px; float: left; border: 1px solid black;' src='{s}' />"
                     for s in images ])
    display(HTML(imagesList))

In [111]:
similar_images('laptop/image_0027.jpg')

In [112]:
similar_images('strawberry/image_0027.jpg')

In [106]:
similar_images('water_lilly/image_0009.jpg')